In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb

pd.options.display.max_rows = 25

In [3]:

IDIR = '../data/'


print('loading prior')
priors = pd.read_csv(IDIR + 'order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading train')
train = pd.read_csv(IDIR + 'order_products__train.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading orders')
orders = pd.read_csv(IDIR + 'orders.csv', dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})

print('loading products')
products = pd.read_csv(IDIR + 'products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8},
        usecols=['product_id', 'aisle_id', 'department_id'])

print('priors {}: {}'.format(priors.shape, ', '.join(priors.columns)))
print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))
print('train {}: {}'.format(train.shape, ', '.join(train.columns)))

###

print('computing product f')
prods = pd.DataFrame()
prods['orders'] = priors.groupby(priors.product_id).size().astype(np.int32)
prods['reorders'] = priors['reordered'].groupby(priors.product_id).sum().astype(np.float32)
prods['reorder_rate'] = (prods.reorders / prods.orders).astype(np.float32)
products = products.join(prods, on='product_id')
products.set_index('product_id', drop=False, inplace=True)
del prods


print('add order info to priors')
orders.set_index('order_id', inplace=True, drop=False)
priors = priors.join(orders, on='order_id', rsuffix='_')
priors.drop('order_id_', inplace=True, axis=1)


loading prior
loading train
loading orders
loading products
priors (32434489, 4): order_id, product_id, add_to_cart_order, reordered
orders (3421083, 7): order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order
train (1384617, 4): order_id, product_id, add_to_cart_order, reordered
computing product f
add order info to priors


In [16]:
print(orders[orders.eval_set == 'train'].order_id[:5])
train[train.order_id == 1187899]

order_id
1187899    1187899
1492625    1492625
2196797    2196797
525192      525192
880375      880375
Name: order_id, dtype: int32


,order_id,product_id,add_to_cart_order,reordered
484420,1187899,196,1,1
484421,1187899,25133,2,1
484422,1187899,38928,3,1
484423,1187899,26405,4,1
484424,1187899,39657,5,1
484425,1187899,10258,6,1
484426,1187899,13032,7,1
484427,1187899,26088,8,1
484428,1187899,27845,9,0
484429,1187899,49235,10,1


In [3]:
products.head()

,product_id,aisle_id,department_id,orders,reorders,reorder_rate
product_id,,,,,,
1,1,61,19,1852.0,1136.0,0.613391
2,2,104,13,90.0,12.0,0.133333
3,3,94,7,277.0,203.0,0.732852
4,4,38,1,329.0,147.0,0.446809
5,5,5,13,15.0,9.0,0.600000


In [4]:
t = globals()
t.keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', '_sh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'np', 'pd', 'lgb', '_i2', 'IDIR', 'priors', 'train', 'orders', 'products', '_i3', '_3', '_i4', 't'])

In [5]:
priors['reordered'].groupby(priors.product_id).sum().astype(np.float32)

product_id
1         1136.0
2           12.0
3          203.0
4          147.0
5            9.0
6            3.0
7           12.0
8           83.0
9           82.0
10        1304.0
11          59.0
12         126.0
          ...   
49677       72.0
49678      149.0
49679       40.0
49680      300.0
49681       39.0
49682       38.0
49683    67313.0
49684        1.0
49685        6.0
49686       84.0
49687        6.0
49688       15.0
Name: reordered, Length: 49677, dtype: float32

In [15]:
priors.groupby(priors.product_id).size()

product_id
1         1852
2           90
3          277
4          329
5           15
6            8
7           30
8          165
9          156
10        2572
11         104
12         246
         ...  
49677      221
49678      377
49679      132
49680     1018
49681       70
49682      108
49683    97315
49684        9
49685       49
49686      120
49687       13
49688       89
Length: 49677, dtype: int64

In [7]:
print('computing user f')
usr = pd.DataFrame()
usr['average_days_between_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
usr['nb_orders'] = orders.groupby('user_id').size().astype(np.int16)

users = pd.DataFrame()
users['total_items'] = priors.groupby('user_id').size().astype(np.int16)
users['all_products'] = priors.groupby('user_id')['product_id'].apply(set)
users['total_distinct_items'] = (users.all_products.map(len)).astype(np.int16)

users = users.join(usr)


computing user f


In [8]:
users['total_distinct_items']

user_id
1          18
2         102
3          33
4          17
5          23
6          12
7          68
8          36
9          58
10         94
11         61
12         61
         ... 
206198     35
206199    134
206200     95
206201    174
206202     74
206203     84
206204     43
206205     24
206206    150
206207     92
206208    198
206209     68
Name: total_distinct_items, Length: 206209, dtype: int16

In [9]:
priors.product_id + priors.user_id * 100000

0           20227933120
1           20227928985
2           20227909327
3           20227945918
4           20227930035
5           20227917794
6           20227940141
7           20227901819
8           20227943668
9           20597033754
10          20597024838
11          20597017704
               ...     
32434477    17518512023
32434478    17518547941
32434479     2524707854
32434480     2524745309
32434481     2524721162
32434482     2524718176
32434483     2524735211
32434484     2524739678
32434485     2524711352
32434486     2524704600
32434487     2524724852
32434488     2524705020
Length: 32434489, dtype: int64

In [10]:
priors.product_id

0           33120
1           28985
2            9327
3           45918
4           30035
5           17794
6           40141
7            1819
8           43668
9           33754
10          24838
11          17704
            ...  
32434477    12023
32434478    47941
32434479     7854
32434480    45309
32434481    21162
32434482    18176
32434483    35211
32434484    39678
32434485    11352
32434486     4600
32434487    24852
32434488     5020
Name: product_id, Length: 32434489, dtype: uint16

In [11]:
priors['user_product'] = priors.product_id + priors.user_id * 100000

In [12]:
priors.user_product

0           20227933120
1           20227928985
2           20227909327
3           20227945918
4           20227930035
5           20227917794
6           20227940141
7           20227901819
8           20227943668
9           20597033754
10          20597024838
11          20597017704
               ...     
32434477    17518512023
32434478    17518547941
32434479     2524707854
32434480     2524745309
32434481     2524721162
32434482     2524718176
32434483     2524735211
32434484     2524739678
32434485     2524711352
32434486     2524704600
32434487     2524724852
32434488     2524705020
Name: user_product, Length: 32434489, dtype: int64

In [13]:
d= dict()
for row in priors.itertuples():
    z = row.user_product
    if z not in d:
        d[z] = (1,
                (row.order_number, row.order_id),
                row.add_to_cart_order)
    else:
        d[z] = (d[z][0] + 1,
                max(d[z][1], (row.order_number, row.order_id)),
                d[z][2] + row.add_to_cart_order)

In [18]:
max(d[z][1], (row.order_number, row.order_id))


(26, 1929747)

In [ ]:
d[15612224773]

In [30]:
userXproduct = pd.DataFrame.from_dict(d, orient='index')

In [31]:
userXproduct.columns = ['nb_orders', 'last_order_id', 'sum_pos_in_cart']

In [32]:
userXproduct.columns = ['nb_orders', 'last_order_id', 'sum_pos_in_cart']
userXproduct.nb_orders = userXproduct.nb_orders.astype(np.int16)
userXproduct.last_order_id = userXproduct.last_order_id.map(lambda x: x[1]).astype(np.int32)
userXproduct.sum_pos_in_cart = userXproduct.sum_pos_in_cart.astype(np.int16)
print('user X product f', len(userXproduct))

user X product f 13307953


In [33]:
userXproduct

,nb_orders,last_order_id,sum_pos_in_cart
20227933120,5,104690,10
20227928985,5,132412,16
20227909327,1,2,3
20227945918,5,2382766,24
20227930035,3,2382766,14
20227917794,7,2382766,25
20227940141,5,132412,29
20227901819,2,2,19
20227943668,3,132412,20
20597033754,17,368699,86


In [ ]:
userXproduct

In [21]:
train_orders = orders[orders.eval_set == 'train']
train.set_index(['order_id', 'product_id'], inplace=True, drop=False)

In [27]:
(146428, 146428) in train.index[:10]

False

In [35]:
train_orders.columns

Index(['order_id', 'user_id', 'eval_set', 'order_number', 'order_dow',
       'order_hour_of_day', 'days_since_prior_order'],
      dtype='object')